# 配置虚拟环境+导入相关库

创建了一个虚拟环境，这样某些相关库不会报错
相关配置见 requirement.txt

In [1]:
import sys
print(sys.executable)

e:\study\CASAterm2\CASA0003\groupwork\coffeestory\.venv\Scripts\python.exe


In [2]:
import osmnx as ox
import geopandas as gpd

In [3]:
import pandas as pd

# get poi

In [3]:
# 定义城市列表
cities = {
    "Tokyo": "Tokyo, Japan",
    "Melbourne": "Melbourne, Australia",
    "Rome": "Rome, Italy",
    "Guangzhou": "Guangzhou, China"
}

In [4]:
for city_name, city_query in cities.items():
    print(f"Fetching cafes in {city_name}...")
    
    cafes = ox.geometries_from_place(city_query, tags={"amenity": "cafe"})
    cafes = cafes[["name", "geometry", "amenity"]].copy()
    cafes["city"] = city_name

    cafes.to_file(f"{city_name.lower()}_cafes.geojson", driver="GeoJSON")

print("All city POI saved!")

Fetching cafes in Tokyo...


C:\Users\10851\AppData\Local\Temp\ipykernel_27476\30475318.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  cafes = ox.geometries_from_place(city_query, tags={"amenity": "cafe"})


Fetching cafes in Melbourne...


C:\Users\10851\AppData\Local\Temp\ipykernel_27476\30475318.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  cafes = ox.geometries_from_place(city_query, tags={"amenity": "cafe"})


Fetching cafes in Rome...


C:\Users\10851\AppData\Local\Temp\ipykernel_27476\30475318.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  cafes = ox.geometries_from_place(city_query, tags={"amenity": "cafe"})


Fetching cafes in Guangzhou...


C:\Users\10851\AppData\Local\Temp\ipykernel_27476\30475318.py:4: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  cafes = ox.geometries_from_place(city_query, tags={"amenity": "cafe"})


All city POI saved!


In [9]:
# 文件路径字典
files = {
    "Tokyo": "tokyo_cafes.geojson",
    "Melbourne": "melbourne_cafes.geojson",
    "Rome": "rome_cafes.geojson",
    "Guangzhou": "guangzhou_cafes.geojson"
}

# 初始化结果容器
geometry_summary = {}

# 遍历每个城市并统计几何类型
for city, filepath in files.items():
    gdf = gpd.read_file(filepath)
    geom_counts = gdf.geometry.geom_type.value_counts()
    geometry_summary[city] = geom_counts

# 整理成 DataFrame 展示
summary_df = pd.DataFrame(geometry_summary).fillna(0).astype(int)
print(summary_df)

         Tokyo  Melbourne  Rome  Guangzhou
Point     4557       2521  1989        342
Polygon    144        432   124          9


In [11]:
# 分离 point 和 polygon
points = gdf[gdf.geometry.geom_type == "Point"]
polygons = gdf[gdf.geometry.geom_type == "Polygon"]

points = points.to_crs(epsg=4326)
polygons = polygons.to_crs(epsg=4326)

# 找出所有点在某个多边形内的情况
joined = gpd.sjoin(points, polygons, how="inner", predicate="within")

# 输出可能重复或重合的项
print(joined[["name_left", "name_right"]])


Empty DataFrame
Columns: [name_left, name_right]
Index: []


In [12]:
# 筛选 Polygon 类型数据
polygons = gdf[gdf.geometry.geom_type == "Polygon"]

# 查看其属性列（不包含 geometry）和前几行内容
print(polygons.head())


    element_type       osmid    name amenity       city  \
6            way  1237736430     咖啡厅    cafe  Guangzhou   
138          way  1144924594  家·湾区书屋    cafe  Guangzhou   
139          way  1380577122      茶饮    cafe  Guangzhou   
140          way  1380581973   王者奶茶店    cafe  Guangzhou   
145          way   180524998    今啡昔笔    cafe  Guangzhou   

                                              geometry  
6    POLYGON ((113.64734 22.62422, 113.64750 22.624...  
138  POLYGON ((113.33586 23.12288, 113.33586 23.122...  
139  POLYGON ((113.40090 22.95314, 113.40118 22.953...  
140  POLYGON ((113.40482 22.95163, 113.40507 22.951...  
145  POLYGON ((113.48945 23.45412, 113.48959 23.453...  


In [13]:
for city, path in files.items():
    print(f"正在处理：{city}")

    # 读取数据
    gdf = gpd.read_file(path)

    # 分离点与面
    points = gdf[gdf.geometry.geom_type == "Point"]
    polygons = gdf[gdf.geometry.geom_type == "Polygon"].copy()

    # 将面转为质心
    polygons["geometry"] = polygons.centroid

    # 合并为统一 Point 数据集
    unified = pd.concat([points, polygons], ignore_index=True)
    unified = unified.set_geometry("geometry")  # 确保几何列正确
    unified = unified.to_crs(epsg=4326)         # 保持统一坐标系

    # 覆盖保存（可加 backup 步骤）
    unified.to_file(path, driver="GeoJSON")

    print(f"✅ {city} 处理完成，共 {len(unified)} 条记录\n")

正在处理：Tokyo


C:\Users\10851\AppData\Local\Temp\ipykernel_25216\1065328155.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["geometry"] = polygons.centroid


✅ Tokyo 处理完成，共 4701 条记录

正在处理：Melbourne


C:\Users\10851\AppData\Local\Temp\ipykernel_25216\1065328155.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["geometry"] = polygons.centroid


✅ Melbourne 处理完成，共 2953 条记录

正在处理：Rome


C:\Users\10851\AppData\Local\Temp\ipykernel_25216\1065328155.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["geometry"] = polygons.centroid


✅ Rome 处理完成，共 2113 条记录

正在处理：Guangzhou
✅ Guangzhou 处理完成，共 351 条记录



C:\Users\10851\AppData\Local\Temp\ipykernel_25216\1065328155.py:12: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["geometry"] = polygons.centroid


# get boundary

In [17]:
# 设置不显示下载时的警告
ox.settings.log_console = True
ox.settings.use_cache = True

# 城市名称（OSM 能识别的格式）
cities = {
    "guangzhou": "Guangzhou, China",
    "melbourne": "Melbourne, Australia",
    "rome": "Rome, Italy",
    "tokyo": "Tokyo, Japan"
}

for city_key, query in cities.items():
    print(f"正在获取 {query} 的数据...")

    # 1. 获取城市边界
    boundary = ox.geocode_to_gdf(query)
    boundary.to_file(f"{city_key}_boundary.geojson", driver="GeoJSON")

    # 2. 获取城市内的行政区（如区/郊区）
    admin = ox.geometries_from_place(query, tags={
        "boundary": "administrative",
        "admin_level": True
    })

    # 可选过滤（保留 admin_level 6–10）
    admin = admin[admin["admin_level"].isin(["6", "7", "8", "9", "10"])]
    admin = admin.to_crs(epsg=4326)
    admin.to_file(f"{city_key}_admin.geojson", driver="GeoJSON")

    print(f"✅ {city_key} 完成：边界 {len(boundary)}, 区划 {len(admin)}\n")

正在获取 Guangzhou, China 的数据...


C:\Users\10851\AppData\Local\Temp\ipykernel_25216\2913309053.py:21: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  admin = ox.geometries_from_place(query, tags={


✅ guangzhou 完成：边界 1, 区划 921

正在获取 Melbourne, Australia 的数据...


C:\Users\10851\AppData\Local\Temp\ipykernel_25216\2913309053.py:21: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  admin = ox.geometries_from_place(query, tags={


✅ melbourne 完成：边界 1, 区划 616

正在获取 Rome, Italy 的数据...


C:\Users\10851\AppData\Local\Temp\ipykernel_25216\2913309053.py:21: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  admin = ox.geometries_from_place(query, tags={


✅ rome 完成：边界 1, 区划 292

正在获取 Tokyo, Japan 的数据...


C:\Users\10851\AppData\Local\Temp\ipykernel_25216\2913309053.py:21: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  admin = ox.geometries_from_place(query, tags={


✅ tokyo 完成：边界 1, 区划 13570

